In [ ]:
import sys # loading commands to control/navigate within the system architecture
# Loading pandas, a library for data manipulation
from os.path import join
# import xlrd
import pandas as pd
# import lxml

# Loading numpy, a library fo manipulation of numbers
import numpy as np

# loading matplotlib, a library for visualization
import matplotlib.pyplot as plt
%matplotlib inline

from biolabsim import Host, Strain, Ecol
from biolabsim import measure_EnzymeLevel1, Help_GenomeGenerator, Strain, make_UpdateExpression, Help_StrainCharacterizer, Help_PromoterStrength, Help_Expr2Flux
from Bio.Seq import Seq
from pandas.core.frame import DataFrame
from copy import deepcopy

In [ ]:
def add_TranscriptionFactor(df, RegID: str, act_rep: str, PromSeq: str, ORFSeq: str):
    newdf = df

    # check if TF is already present, add columns to the default TF accordingly, no zero initiation of RegType column
    if any('TF_regulated' in s for s in newdf.columns.values):
        Flux = 10
        Expr = Help_PromoterStrength('Ecol', PromSeq)
        TF = {
            'RctID': [RegID],
            'Expression': [Expr],  # needs to be != 0
            'Promoter': [PromSeq],  # better be unique
            'ORF': [ORFSeq],  # must be unique
            'Fluxes': [Flux],  # value not further relevant
            'Expr2Flux':  [Help_Expr2Flux(Flux,Expr)],
            'TF_regulated': [0],
            'RegType': act_rep
        }
        dfTF = pd.DataFrame(TF)
        newdf = newdf.append(dfTF, ignore_index=True)

    else:
        # add TF row:
        Flux = 10
        Expr = Help_PromoterStrength('Ecol', PromSeq)
        TF = {
            'RctID': [RegID],
            'Expression': [Expr],
            'Promoter': [PromSeq],
            'ORF': [ORFSeq],
            'Fluxes': [Flux],
            'Expr2Flux': [Help_Expr2Flux(Flux,Expr)]
        }

        dfTF = pd.DataFrame(TF)
        newdf = newdf.append(dfTF, ignore_index=True)

        # add TF_regulated and RegType column:
        zeroarray = np.zeros(len(newdf), dtype=int)
        newdf = newdf.assign(TF_regulated=zeroarray)  
        newdf = newdf.assign(RegType=zeroarray)

        TF_Indx = (newdf['RctID'] == RegID)  # access TF row
        newdf.loc[TF_Indx, 'RegType'] = act_rep  
        
    return newdf

In [ ]:
def add_TranscriptionFactorReg(df, RegID: str, EnzymeTarget: list):
    newdf = df
    # set Regulation of some Proteins:
    for ETarget in EnzymeTarget:
        # Future: fct recognizes what substrates are there, then makes TF_act or TF_rep
        # define new and changed functions

        myIndx = (newdf['RctID'] == ETarget)
        newdf.loc[myIndx, 'TF_regulated'] = RegID

    return newdf 

In [ ]:
def Update_DF(df):
    # update expression and Expr2Flux
    corrExpr = calculate_StandardRegulatedExpression(df)
    df['Expression'] = corrExpr
    # update for Expression to Flux correlation, for all a linear factor
    myExpr2Flux = Help_Expr2Flux(df['Fluxes'], corrExpr)
    df['Expr2Flux'] = myExpr2Flux
    return df

In [ ]:
def Help_mutatePromoter(Mutant_Targets: list, wtStrain, mtStrain, insertSeq: str, cutSeq: str ='ATTGA'):
    # Mutants to be generated

    for myMutant in Mutant_Targets:

        MTdf = mtStrain.genes_df.copy()
        Mutant_Indx = tuple(MTdf[MTdf['RctID'] == myMutant].index.values)
        # Finding the index for later printing
        WTIndx = str(wtStrain.genome).find(wtStrain.genes_df.loc[Mutant_Indx, 'Promoter'])
    #     # Generating target promoter sequence, ATTGA is always there
        PromTar = MTdf.loc[Mutant_Indx, 'Promoter'].replace(cutSeq,insertSeq)
        # converting Biopython Seq class to string. This makes string replacements easier
        MutGenome = str(mtStrain.genome)
        MutGenome = MutGenome.replace(MTdf.loc[Mutant_Indx, 'Promoter'], PromTar)
        MTdf.loc[Mutant_Indx, 'Promoter'] = PromTar
    #     print('{}, Reference:\t{},\nMutated:\t{}\nwith index {}'.format(myMutant, wtHost.strain.genome[Mutant_Indx:Mutant_Indx+41], myHost.strain.genome[Mutant_Indx:Mutant_Indx+41], Mutant_Indx))
        print(myMutant, WTIndx)
    

        
    return MutGenome, MTdf

In [ ]:
def make_DetectRegulatorPromoterMut(StrainMut, RctNewDF):
    #     for obj in (StrainWt, StrainMut):
    #         assert isinstance(obj, Strain), 'wrong strain type: must be Strain'
    #     assert isinstance(RctNewDF, DataFrame), 'ReactNewDF of wrong type: input dataframe'
    GenesDF = StrainMut.genes_df
    # finding regulators in GenesDF
    myRegIndx = GenesDF[GenesDF['RegType'] != 0].index.values
    # finding index of regulators with changes in the promoter
    RegUpdatePromIndx = myRegIndx[RctNewDF.loc[myRegIndx, 'RctFlag'].values]
    # finding name of regulators with changes in the promoter
    RegUpdatePromName = RctNewDF.loc[RegUpdatePromIndx, 'RctID'].values
    # finding enzymes regulated by regulator
    EnzUpdateReguIndx = GenesDF['TF_regulated'].isin(RegUpdatePromName)

    return EnzUpdateReguIndx

In [ ]:
def measure_EnzymeLevel(HostName: str, StrainWT, StrainMut):
    '''
    The function differences of expression levels of enzymes between two strains.
    '''
    RefGenDF = StrainWT.genes_df
    RefGenome = str(StrainWT.genome)
    MutGenome = str(StrainMut.genome)
    RefModel = StrainWT.model

    RctNewDF = Help_StrainCharacterizer(HostName, RefGenDF, RefGenome, MutGenome, RefModel)

    # identifying changes in regulator expression
    RegNewAr = make_DetectRegulatorPromoterMut(StrainMut, RctNewDF)

    # combining promoter changes enzyme + regulator
    AllNewAr = RctNewDF['RctFlag'].values | RegNewAr
    AllNewDF = deepcopy(RctNewDF)
    AllNewDF['RctFlag'] = AllNewAr

    return AllNewDF

In [1]:
def calculate_StandardRegulatedExpression(WTdf):
    # finding the different regulator types and their equations which have their promoter changed
    RegTypes = [i for i in list(set(WTdf['TF_regulated'].values))]
    # removing the non regulated elements
    RegTypes.remove(0)
    print(RegTypes)


    for RegType in RegTypes:
        EnzIndx = WTdf['TF_regulated'] == RegType
        RegIndx = WTdf['RctID'] == RegType
        EquType = WTdf.loc[WTdf['RctID'] == RegType, 'RegType'].values
             
        MaxExpr = 2 * WTdf.loc[EnzIndx, 'Expression'].values  # bei physiologischem cTF-Wert Flux= mtExpr
        cTF = WTdf.loc[RegIndx, 'Expression'].values  # abhängig von geändertem TF-Promotor
        K = WTdf.loc[EnzIndx, 'Expression'].values  # physiologischer Wert von c, Expression/Expr2Flux, beeinflusst Aktivität des TF, WT:nicht durch promotor geändert
        n = 1.95  # Kim, Harold D.; O'Shea, Erin K. (2008): A quantitative model of transcription factor–activated gene expression. DOI: 10.1038/nsmb.1500.
        #         print ('Eqn: ', RegType, MaxExpr, cTF, K)

        if EquType == 'Activator':
            corrExpr = MaxExpr / (1 + (K / cTF) ** n)

        elif EquType == 'Repressor':
            corrExpr = MaxExpr * (1 - 1 / (1 + (K / cTF) ** n))

        PreProcFlux = deepcopy(WTdf['Expression'].values)
        PreProcFlux[EnzIndx] = corrExpr

    return PreProcFlux

In [ ]:
def calculate_mutatedRegulatedExpression(WTdf, AllNewDF):
    # finding the different regulator types and their equations which have their promoter changed
    RegTypes = [i for i in list(set(WTdf.loc[AllNewDF['RctFlag'], 'TF_regulated'].values))]
    # removing the non regulated elements
    RegTypes.remove(0)

    for RegType in RegTypes:
        EnzIndx = WTdf['TF_regulated'] == RegType
        RegIndx = WTdf['RctID'] == RegType
        EquType = WTdf.loc[WTdf['RctID'] == RegType, 'RegType'].values
        
        MaxExpr = 2 * AllNewDF.loc[EnzIndx, 'RefExpr'].values  # bei physiologischem cTF-Wert Flux= mtExpr
        cTF = AllNewDF.loc[RegIndx, 'NewExpr'].values  # abhängig von geändertem TF-Promotor
        K = AllNewDF.loc[EnzIndx, 'RefExpr'].values  # physiologischer Wert von c, Expression/Expr2Flux, beeinflusst Aktivität des TF, WT:nicht durch promotor geändert
        n = 1.95  # Kim, Harold D.; O'Shea, Erin K. (2008): A quantitative model of transcription factor–activated gene expression. DOI: 10.1038/nsmb.1500.
        #         print ('Eqn: ', AllNewDF)

        if EquType == 'Activator':
            corrExpr = MaxExpr / (1 + (K / cTF) ** n)

        elif EquType == 'Repressor':
            corrExpr = MaxExpr * (1 - 1 / (1 + (K / cTF) ** n))

        PreProcFlux = AllNewDF['NewExpr'].values
        PreProcFlux[EnzIndx] = corrExpr

    return PreProcFlux

In [ ]:
def setboundary(df, AllNewDF):
    '''
    Finding and setting the boundaries for cobrapy.
    '''

    # finding reactions for which the expression has changed
    OnlyEnzIndx = df['RegType'] == 0
    RctNewDF = AllNewDF.loc[OnlyEnzIndx, ['RctFlag']] == True
    RctNew = np.arange((len(RctNewDF)))[np.ravel(RctNewDF.values)]
    #     OnlyEnzIndx = WTdf[WTdf['RegType'] == 0].index.values
    #     RctNew = AllNewDF[AllNewDF.loc[OnlyEnzIndx, 'RctFlag'] == True].index.values

    # For reactions with reduced and positive flux: reduce the upper limit,
    # For reactions with increased and positive flux: increase the lower limit
    # For reactions with negative flux the limits are exchanged.
    FluxPos = RctNew[tuple([AllNewDF.loc[RctNew, 'RefFlux'] > 0])]
    FluxNeg = RctNew[tuple([AllNewDF.loc[RctNew, 'RefFlux'] < 0])]
    # Finding increased and decreased fluxes
    FluxInc = RctNew[AllNewDF.loc[RctNew, 'NewFlux'].values / AllNewDF.loc[RctNew, 'RefFlux'].values > 1]
    FluxDec = RctNew[AllNewDF.loc[RctNew, 'NewFlux'].values / AllNewDF.loc[RctNew, 'RefFlux'].values < 1]

    # Comb.1: positive flux with increased expression -> increasing lower bound
    PosIncInd = np.intersect1d(FluxPos, FluxInc)
    # Comb.2: positive flux with decreased expression -> decreasing upper bound
    PosDecInd = np.intersect1d(FluxPos, FluxDec)
    # Comb.3: negative flux with increased expression -> decreasing lower bound
    NegIncInd = np.intersect1d(FluxNeg, FluxInc)
    # Comb.4: positive flux with increased expression -> increasing lower bound
    NegDecInd = np.intersect1d(FluxNeg, FluxDec)

    Expr_Change = {'increase': np.hstack([PosIncInd, NegIncInd]), 'decrease': np.hstack([PosDecInd, NegDecInd])}
    Set_Boundary = {'lower': np.hstack([PosIncInd, NegDecInd]), 'upper': np.hstack([PosDecInd, NegIncInd])}

    return Set_Boundary

In [ ]:
# Finding reactions to change. Three possibilities exits: 1. the promoter of the enzyme itself has changed, 2. the promoter of a regulator has changed, 3. the enzyme promoter and the regulator promoter have changed
# redefining Help_FluxCalculator in simulation/metabolism
def Help_FluxCalculator(HostName: str, Strain, AllNewDF=None):
    '''
    Calculation of flux values.

    This method can work in 2 modes:
      [Strain only] : The metabolic model of the WT strain is used for calculation.
      [Strain + AllNewDF] : An additional step of resetting boundaries is done on the model
        before the fluxes are calculated.

    TODO: The "reset boundary" step is mutating the `StrainMut.model`, mutation might not be intended.
      Because of this, perhaps the other `Strain.model` gets mutated in the process.
    '''
    #     from ..measurement.fluxes import measure_EnzymeLevel1

    # adding flux values
    # setup of flux boundaries. For the reference boundary changes are set to 'False',
    # for mutant strains, ractions with altered promoter sequence will change enzyme levels and boundaries must be changed accordingly, their variable is 'True'

    if AllNewDF is not None:
        print('resetting boundaries')
        # finding reactions for which the expression has changed, and finding the new level
        RefFlux = AllNewDF['RefFlux']
        NewFlux = AllNewDF['NewFlux']

        Set_Boundary = setboundary(Strain.genes_df, AllNewDF)
        # Defining the model with the two combinations of either
        # increasing lower bound (increased forward, decreased reverse reaction)
        # decreasing upper bound (decreased forward, increased reverse reaction)
        with Strain.model as myModel:
            # Comb.1: positive flux with increased expression -> increasing lower bound
            for Indx in Set_Boundary['lower']:
                myModel.reactions[Indx].lower_bound = AllNewDF.loc[Indx, 'NewFlux']
            # Comb.2: positive flux with decreased expression -> decreasing upper bound
            for Indx in Set_Boundary['upper']:
                myModel.reactions[Indx].upper_bound = AllNewDF.loc[Indx, 'NewFlux']

            Fluxes = myModel.optimize()

    else:
        Fluxes = Strain.model.optimize()

    return Fluxes.fluxes.values, Fluxes.objective_value